# Matching citation counts from Aminer v14

Author: Natkamon Tovanich
Last Update: 06/06/2024

**Instructions**
1. Run first_scan.ipynb to match the paper title and/or DOI with Aminer
2. Run second_scan.py to find the potential match papers from string distance function
3. Manually select the match papers from the second scan in candidate_papers.csv
4. Run merge_data.ipynb to match the result from the first (exact title or DOI match) and second scan (string distance candidates).
5. The result is available at vispubdata_citation.csv.

### First scan: match by DOI or exact title

In [1]:
import pandas as pd
import json
import re
import pickle

Load the datasets

At the moment you have to switch datasets manually here. We need to fix this later so it runs through both automatically.

In [11]:
datasets = ['./vispubdata-update/results/vispubdata-update.csv',
            './vispubdata-update/results/vispubdata-update-journals.csv']
appendices = ['vis-papers','journal-papers']

datasets_dfs = []

for dataset in datasets:
    df = pd.read_csv(dataset, keep_default_na=False)
    datasets_dfs.append(df)
    


,Conference,Year,Title,DOI,Link,FirstPage,LastPage,PaperType,Abstract,AuthorNames-Deduped,AuthorNames,AuthorAffiliation,InternalReferences,AuthorKeywords,AminerCitationCount,CitationCount_CrossRef,PubsCited_CrossRef,Downloads_Xplore,Award,GraphicsReplicabilityStamp
0,Vis,2022,Photosensitive Accessibility for Interactive D...,10.1109/tvcg.2022.3209359,http://dx.doi.org/10.1109/TVCG.2022.3209359,374,384,J,Accessibility guidelines place restrictions on...,Laura South;Michelle A. Borkin,Laura South;Michelle A. Borkin,"Northeastern University, USA;Northeastern Univ...",.1109/tvcg.2011.185;10.1109/tvcg.2021.3114829;...,"accessibility,photosensitive epilepsy,photosen...",,4,63,579,,
1,Vis,2022,HetVis: A Visual Analysis Approach for Identif...,10.1109/tvcg.2022.3209347,http://dx.doi.org/10.1109/TVCG.2022.3209347,310,319,J,Horizontal federated learning (HFL) enables di...,Xumeng Wang;Wei Chen 0001;Jiazhi Xia;Zhen Wen;...,Xumeng Wang;Wei Chen;Jiazhi Xia;Zhen Wen;Rongc...,"TMCC, CS, Nankai University, China;State Key L...",.1109/tvcg.2015.2467618;10.1109/tvcg.2019.2934...,"Federated learning,data heterogeneity,cluster ...",,11,43,1038,,
2,Vis,2022,Rigel: Transforming Tabular Data by Declarativ...,10.1109/tvcg.2022.3209385,http://dx.doi.org/10.1109/TVCG.2022.3209385,128,138,J,"We present Rigel, an interactive system for ra...",Ran Chen;Di Weng;Yanwei Huang;Xinhuan Shu;Jiay...,Ran Chen;Di Weng;Yanwei Huang;Xinhuan Shu;Jiay...,"State Key Lab of CAD&CG, Zhejiang University, ...",.1109/tvcg.2021.3114830;10.1109/vast47406.2019...,"Data transformation,self-service data transfor...",,6,68,627,,
3,Vis,2022,BeauVis: A Validated Scale for Measuring the A...,10.1109/tvcg.2022.3209390,http://dx.doi.org/10.1109/TVCG.2022.3209390,363,373,J,We developed and validated a rating scale to a...,Tingying He;Petra Isenberg;Raimund Dachselt;To...,Tingying He;Petra Isenberg;Raimund Dachselt;To...,"Université Paris-Saclay, CNRS, Inria, LISN, Fr...",.1109/infvis.2005.1532128;10.1109/tvcg.2006.18...,"Aesthetics,aesthetic pleasure,validated scale,...",,7,79,784,,X
4,Vis,2022,NAS-Navigator: Visual Steering for Explainable...,10.1109/tvcg.2022.3209361,http://dx.doi.org/10.1109/TVCG.2022.3209361,299,309,J,The success of DL can be attributed to hours o...,Anjul Kumar Tyagi;Cong Xie;Klaus Mueller 0001,Anjul Tyagi;Cong Xie;Klaus Mueller,"Computer Science Department, Visual Analytics ...",.1109/vast.2012.6400490;10.1109/tvcg.2019.2934...,"Deep Learning,Neural Network Architecture Sear...",,2,63,412,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3748,Vis,2023,Extract and Characterize Hairpin Vortices in T...,10.1109/tvcg.2023.3326603,http://dx.doi.org/10.1109/TVCG.2023.3326603,716,726,J,Hairpin vortices are one of the most important...,Adeel Zafar;Di Yang;Guoning Chen,Adeel Zafar;Di Yang;Guoning Chen,"University of Houston, USA;University of Houst...",0.1109/visual.1994.346327;10.1109/tvcg.2018.28...,"Turbulent flow,vortices,hairpin vortex extraction",,0,55,199,HM,
3749,Vis,2023,Causality-Based Visual Analysis of Questionnai...,10.1109/tvcg.2023.3327376,http://dx.doi.org/10.1109/TVCG.2023.3327376,638,648,J,"As the final stage of questionnaire analysis, ...",Renzhong Li;Weiwei Cui;Tianqi Song;Xiao Xie;Ru...,Renzhong Li;Weiwei Cui;Tianqi Song;Xiao Xie;Ru...,"State Key Lab of CAD&CG, Zhejiang University, ...",0.1109/tvcg.2021.3114875;10.1109/tvcg.2022.320...,"Causal analysis,Questionnaire,Design study",,0,44,352,,
3750,Vis,2023,Quantivine: A Visualization Approach for Large...,10.1109/tvcg.2023.3327148,http://dx.doi.org/10.1109/TVCG.2023.3327148,573,583,J,Quantum computing is a rapidly evolving field ...,Zhen Wen;Yihan Liu;Siwei Tan;Jieyi Chen;Minfen...,Zhen Wen;Yihan Liu;Siwei Tan;Jieyi Chen;Minfen...,"State Key Lab of CAD&CG, Zhejiang University, ...",0.1109/tvcg.2018.2865139;10.1109/infvis.2004.1...,"Quantum circuit,semantic analysis,visual abstr...",,0,83,227,,
3751,Vis,2023,The Arrangement of Marks Impacts Afforded Mess...,10.1109/tv

Convert title to lower case

In [6]:
for df in datasets_dfs:
    df["title"] = df["Title"].apply(lambda x: re.sub(r'[^\w]', '', str(x).lower()))
    df["doi"] = df["DOI"].apply(lambda x: str(x).lower())

Read AMiner v14 dataset line by line and check if the title is matched.

In [8]:
%%time
def checkTitleMatch(titles,dois):
    
    count, match_doi, match_title = 0, 0, 0
    m, l, choices = list(), dict(), dict()
    with open("dblp_v14.json", encoding="utf8") as infile:
        for line in infile:
            line = line.strip().strip(',').strip("]'")
            if line[0] != '{':
                continue
            paper = json.loads(line)
            lower = re.sub(r'[^\w]', '', str(paper['title']).lower())
            
            # Add all papers title to check for missing match
            choices[paper['title'].lower()] = paper['id']
            
            # First, try to match by paper DOI
            if "doi" in paper and str(paper["doi"]).lower() in dois:
                m.append([dois[str(paper["doi"]).lower()], paper['id'], 'doi'])
                l[paper['id']] = paper
                match_doi += 1
            
            # Match by the exact title
            elif lower in titles:
                m.append([titles[lower], paper['id'], 'title'])
                l[paper['id']] = paper
                match_title += 1
                
            count += 1 
            if (count % 1000000 == 0):
                print(count, match_doi, match_title)
        print(count, match_doi, match_title)
    return m #TODO check if we can return two things

FileNotFoundError: [Errno 2] No such file or directory: 'dblp_v14.json'

Convert to dictionary for the faster check

In [7]:
index = 0
for df in datasets_dfs:
    titles = df.set_index('title')['DOI'].to_dict()
    dois = df.set_index('doi')['DOI'].to_dict()
    m = checkTitleMatch(titles,dois)
    m = pd.DataFrame(m, columns=['vispub_doi', 'aminer_id', 'method'])
    m.to_csv('results/exact_matching'+appendices[index]+".csv", index=False)

    with open('results/aminer_titles'+appendices[index]+".p", 'wb') as fp:
        pickle.dump(choices, fp, protocol=pickle.HIGHEST_PROTOCOL)